# Raising Errors

In [1]:
-- View a system message
INSERT INTO SalesLT.SalesOrderDetail (SalesOrderID, OrderQty, ProductID, UnitPrice, UnitPriceDiscount)
VALUES (7617852, 1, 680, 1431.50, 0.00);

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the FOREIGN KEY constraint "FK_SalesOrderDetail_SalesOrderHeader_SalesOrderID". The conflict occurred in database "AdventureWorksLT", table "SalesLT.SalesOrderHeader", column 'SalesOrderID'.

The statement has been terminated.

Total execution time: 00:00:00.083

In [4]:
-- Raise error with RAISERROR
UPDATE SalesLT.Product
SET DiscontinuedDate = GETDATE()
WHERE ProductID = 8261372388;

IF @@ROWCOUNT < 1
    RAISERROR('No product with that ProductID was found. No products updated.', 16, 0);

(0 rows affected)

: Msg 50000, Level 16, State 0, Line 7
No product with that ProductID was found. No products updated.

Total execution time: 00:00:00.004

In [6]:
-- Raise error with THROW
UPDATE SalesLT.Product
SET DiscontinuedDate = GETDATE()
WHERE ProductID = 8261372388;

IF @@ROWCOUNT < 1
    THROW 50001, 'No product with that ProductID was found. No products updated.', 0;

(0 rows affected)

: Msg 50001, Level 16, State 0, Line 7
No product with that ProductID was found. No products updated.

Total execution time: 00:00:00.004

# Catching and Handling Errors

In [13]:
-- Catch and error
BEGIN TRY
    UPDATE SalesLT.Product
    SET ProductNumber = ProductID / ISNULL(Weight, 0);
END TRY
BEGIN CATCH
    PRINT 'The following error occured: ' + ERROR_MESSAGE();
END CATCH

(0 rows affected)

The following error occured: Divide by zero error encountered.

Total execution time: 00:00:00.010

In [15]:
-- Catch and rethrow
BEGIN TRY
    UPDATE SalesLT.Product
    SET ProductNumber = ProductID / ISNULL(Weight, 0);
END TRY
BEGIN CATCH
    PRINT ERROR_MESSAGE();
    THROW;  -- THROW (without parameters) throws last error message
END CATCH

(0 rows affected)

Divide by zero error encountered.

: Msg 8134, Level 16, State 1, Line 3
Divide by zero error encountered.

Total execution time: 00:00:00.008

In [19]:
-- Catch, Log, and Throw a custom error
BEGIN TRY
    UPDATE SalesLT.Product
    SET ProductNumber = ProductID / ISNULL(Weight, 0);
END TRY
BEGIN CATCH
    DECLARE @ErrorLogID AS INT, @ErrorMessage AS VARCHAR(250);
    EXECUTE dbo.uspLogError @ErrorLogID OUTPUT;
    SET @ErrorMessage = 'The update failed because of an error. View error #' + CAST(@ErrorLogID AS varchar)
        + ' in dbo.ErrorLog for details.';
    THROW 500001, @ErrorMessage, 0;
END CATCH

GO 

-- View the error log
SELECT * FROM dbo.ErrorLog;

(0 rows affected)

: Msg 500001, Level 16, State 0, Line 11
The update failed because of an error. View error #3 in dbo.ErrorLog for details.

(1 row affected)

Total execution time: 00:00:00.011

ErrorLogID,ErrorTime,UserName,ErrorNumber,ErrorSeverity,ErrorState,ErrorProcedure,ErrorLine,ErrorMessage
3,2020-04-25 18:40:57.577,dbo,8134,16,1,NULL,3,Divide by zero error encountered.


# Transactions

In [25]:
-- No transaction
BEGIN TRY
    INSERT INTO SalesLT.SalesOrderHeader (DueDate, CustomerID, ShipMethod)
    VALUES (DATEADD(dd, 7, GETDATE()), 1, 'STD DELIVERY');

    DECLARE @SalesOrderID INT = SCOPE_IDENTITY();

    INSERT INTO SalesLT.SalesOrderDetail (SalesOrderID, OrderQty, ProductID, UnitPrice, UnitPriceDiscount)
    VALUES (@SalesOrderID, 1, 999999, 1431.50, 0.00);   -- No product with that ProductID in Product table
END TRY
BEGIN CATCH
    PRINT ERROR_MESSAGE();
END CATCH

(1 row affected)

(0 rows affected)

The INSERT statement conflicted with the FOREIGN KEY constraint "FK_SalesOrderDetail_Product_ProductID". The conflict occurred in database "AdventureWorksLT", table "SalesLT.Product", column 'ProductID'.

Total execution time: 00:00:00.005

In [26]:
-- View orphaned order in SalesOrderHeader with no record in SalesOrderDetail
SELECT soh.SalesOrderID, soh.DueDate, soh.CustomerID, soh.ShipMethod, sod.SalesOrderDetailID
FROM SalesLT.SalesOrderHeader AS soh
LEFT JOIN SalesLT.SalesOrderDetail AS sod
ON soh.SalesOrderID = sod.SalesOrderID
WHERE sod.SalesOrderDetailID IS NULL;

GO 

-- Manually delete that orphaned order in SalesOrderHeader
DELETE FROM SalesLT.SalesOrderHeader
WHERE SalesOrderID = SCOPE_IDENTITY();  -- Gets the last identity (i.e, SalesOrderID) from last query performed (i.e, INSERT INTO SalesLT.SalesOrderHeader)
IF @@ROWCOUNT > 0
    PRINT 'Orphaned order deleted.';

SalesOrderID,DueDate,CustomerID,ShipMethod,SalesOrderDetailID
71954,2020-05-03 09:05:57.920,1,STD DELIVERY,NULL


(1 row affected)

(1 row affected)

Orphaned order deleted.

Total execution time: 00:00:00.008

In [28]:
 -- Using transaction
BEGIN TRY
    BEGIN TRANSACTION
        INSERT INTO SalesLT.SalesOrderHeader (DueDate, CustomerID, ShipMethod)
        VALUES (DATEADD(dd, 7, GETDATE()), 1, 'STD DELIVERY');

        DECLARE @SalesOrderID INT = SCOPE_IDENTITY();

        INSERT INTO SalesLT.SalesOrderDetail (SalesOrderID, OrderQty, ProductID, UnitPrice, UnitPriceDiscount)
        VALUES (@SalesOrderID, 1, 999999, 1431.50, 0.00);   -- No product with that ProductID in Product table
    COMMIT TRANSACTION
 END TRY
 BEGIN CATCH 
    IF @@TRANCOUNT > 0
        BEGIN
            PRINT XACT_STATE();
            ROLLBACK TRANSACTION;
        END
    PRINT ERROR_MESSAGE();
    THROW 50001, 'An insert failed. The transaction was cancelled.', 0;
END CATCH

(1 row affected)

(0 rows affected)

1

The INSERT statement conflicted with the FOREIGN KEY constraint "FK_SalesOrderDetail_Product_ProductID". The conflict occurred in database "AdventureWorksLT", table "SalesLT.Product", column 'ProductID'.

: Msg 50001, Level 16, State 0, Line 20
An insert failed. The transaction was cancelled.

Total execution time: 00:00:00.011

In [29]:
-- View orphaned order in SalesOrderHeader with no record in SalesOrderDetail
SELECT soh.SalesOrderID, soh.DueDate, soh.CustomerID, soh.ShipMethod, sod.SalesOrderDetailID
FROM SalesLT.SalesOrderHeader AS soh
LEFT JOIN SalesLT.SalesOrderDetail AS sod
ON soh.SalesOrderID = sod.SalesOrderID
WHERE sod.SalesOrderDetailID IS NULL;

(0 rows affected)

SalesOrderID,DueDate,CustomerID,ShipMethod,SalesOrderDetailID


Total execution time: 00:00:00.004

In [30]:
-- Use XACT_ABORT
SET XACT_ABORT ON;

BEGIN TRY
    BEGIN TRANSACTION
        INSERT INTO SalesLT.SalesOrderHeader (DueDate, CustomerID, ShipMethod)
        VALUES (DATEADD(dd, 7, GETDATE()), 1, 'STD DELIVERY');

        DECLARE @SalesOrderID INT = SCOPE_IDENTITY();

        INSERT INTO SalesLT.SalesOrderDetail (SalesOrderID, OrderQty, ProductID, UnitPrice, UnitPriceDiscount)
        VALUES (@SalesOrderID, 1, 999999, 1431.50, 0.00);   -- No product with that ProductID in Product table
    COMMIT TRANSACTION
 END TRY
 BEGIN CATCH 
    PRINT ERROR_MESSAGE();
    THROW 50001, 'An insert failed. The transaction was automatically rolled back.', 0;
END CATCH

SET XACT_ABORT OFF;

(1 row affected)

(0 rows affected)

The INSERT statement conflicted with the FOREIGN KEY constraint "FK_SalesOrderDetail_Product_ProductID". The conflict occurred in database "AdventureWorksLT", table "SalesLT.Product", column 'ProductID'.

: Msg 50001, Level 16, State 0, Line 17
An insert failed. The transaction was automatically rolled back.

Total execution time: 00:00:00.011

In [31]:
-- View orphaned order in SalesOrderHeader with no record in SalesOrderDetail
SELECT soh.SalesOrderID, soh.DueDate, soh.CustomerID, soh.ShipMethod, sod.SalesOrderDetailID
FROM SalesLT.SalesOrderHeader AS soh
LEFT JOIN SalesLT.SalesOrderDetail AS sod
ON soh.SalesOrderID = sod.SalesOrderID
WHERE sod.SalesOrderDetailID IS NULL;

(0 rows affected)

SalesOrderID,DueDate,CustomerID,ShipMethod,SalesOrderDetailID


Total execution time: 00:00:00.003